In [1]:
import polars as pl
import json
import statistics
import re
import random
from tqdm.auto import tqdm

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.cluster import AgglomerativeClustering

##### Load data

In [2]:
data_dir = '/Volumes/PortableSSD/CSS/data/processed/'
#data_dir = '/users/ujan/Downloads/'

In [3]:
data_df = pl.read_csv(data_dir+'posts_2015-21_ps_min_2c_politics.csv')
data_df.head()

FileNotFoundError: No such file or directory (os error 2): /Volumes/PortableSSD/CSS/data/processed/posts_2015-21_ps_min_2c_politics.csv

##### Data exploration

In [6]:
# keep id, num_comments etc.
# metaphor match on clusters
    # framebert to remove non metaphors
# engagement in metaphor posts vs non metaphor posts within cluster
    # num comments

##### Filter image domains

In [7]:
no_domains = ["i.imgur.com","imgur.com","i.reddituploads.com","i.sli.mg","i.magaimg.net","gfycat.com","pbs.twimg.com","sli.mg"]

data_df = data_df.filter(~pl.col('domain').is_in(no_domains))

##### Filter images

In [8]:
no_imgs = ['jpeg', 'png', 'tiff', 'gif', 'jpg']

data_df = data_df.filter(~pl.col('url').str.contains_any(no_imgs))

##### Filter small posts

In [9]:
min_chars = 10

data_df = data_df.filter(pl.col('title').str.len_chars() >= min_chars)

##### Cast datetime

In [10]:
MILLISECONDS_IN_SECOND = 1000

datetimes = data_df.select((pl.col("created_utc") * MILLISECONDS_IN_SECOND).cast(
    pl.Datetime).dt.with_time_unit("ms").alias("datetime"))

data_df.replace("created_utc", datetimes['datetime'].dt.date())

/var/folders/by/rktr_w596p97pmt8_cbknvs80000gn/T/ipykernel_6062/928864480.py:4: DeprecationWarning: `with_time_unit` is deprecated. Instead, first cast to `Int64` and then cast to the desired data type.
  pl.Datetime).dt.with_time_unit("ms").alias("datetime"))
/var/folders/by/rktr_w596p97pmt8_cbknvs80000gn/T/ipykernel_6062/928864480.py:6: DeprecationWarning: `replace` is deprecated. DataFrame.replace is deprecated and will be removed in a future version. Please use
    df = df.with_columns(new_column.alias(column_name))
instead.
  data_df.replace("created_utc", datetimes['datetime'].dt.date())


id,created_utc,subreddit,category,super_category,author,domain,url,title,selftext,num_comments,score,gilded,upvote_ratio
str,date,str,str,str,str,str,str,str,str,i64,i64,i64,f64
"""589ygu""",2016-10-19,"""Enough_Sanders…","""politics_2019""","""politics""","""12-juin-3049""","""reddit.com""","""http://www.red…","""Bernout gets e…","""""",12,28,0,100.0
"""58a7z5""",2016-10-19,"""EnoughTrumpSpa…","""politics_2019""","""politics""","""TheIronTARDIS""","""np.reddit.com""","""http://np.redd…","""In case you ne…","""""",38,371,0,100.0
"""58aa2h""",2016-10-19,"""politics""","""politics_2019""","""politics""","""Naggers123""","""breitbart.com""","""http://www.bre…","""Poll: Hillary …","""""",170,305,0,100.0
"""58bnuv""",2016-10-19,"""politics""","""politics_2019""","""politics""","""Metaprinter""","""gop.com""","""https://gop.co…","""Trump asking t…","""""",38,24,0,100.0
"""58c5tw""",2016-10-19,"""ShitRConservat…","""politics_2019""","""politics""","""bluefootedpig""","""reddit.com""","""https://www.re…","""Chab can't und…","""""",2,1,0,100.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2up5gp""",2015-02-04,"""CanadaPolitics…","""politics_2019""","""politics""","""blueberryfickl…","""self.CanadaPol…","""http://www.red…","""Are you curren…","""""",45,10,0,100.0
"""2uvtx0""",2015-02-05,"""Shitstatistssa…","""politics_2019""","""politics""","""highdra""","""investorplace.…","""http://investo…","""The state save…","""""",2,14,0,100.0
"""2uwrz1""",2015-02-05,"""politics""","""politics_2019""","""politics""","""Katalyna""","""cnn.com""","""http://www.cnn…","""What will Puti…","""""",3,6,0,100.0


##### Cluster examples

In [14]:
with open(data_dir+'post_clusters_5.json') as f:
    data = json.load(f)

In [16]:
count = 0

for day, clusters in data.items():
    for cluster_id, post_ids in clusters.items():
        if len(post_ids) > 1:
            print(data_df.filter(pl.col('id').is_in(post_ids))['title'].to_list())
            print('END')
            print(' ')
            count += 1

        if count > 50:
            raise

['Want to streamline government? Start with the Pentagon.', 'WH Petition: reduce the size of the federal government']
END
 
["I've heard people refer to T_D as a safe space. Would you agree or disagree?", 'T_D likes to ban dissenters. How is this different than a safe space?']
END
 
['Jeremy Corbyn uses IBTimes UK video without authorisation to attack Tim Farron', 'Jeremy Corbyn uses IBTimes UK video without authorisation to attack Tim Farron', "Corbyn just pinched our video to attack Tim Farron and didn't even give us credit"]
END
 
['Corbyn responds to labours low poll ratings: "only 3 days ago there was a big swing to Labour in a council by-election"', 'Corbyn responds to labours low poll ratings: "only 3 days ago there was a big swing to Labour in a council by-election"', 'Sophy Ridge: \'"We\'re getting it through": Jeremy Corbyn says voters are getting his message', 'Jeremy Corbyn dodges key question on use of Labour party whips in Brexit vote three times']
END
 
['The inauguratio

RuntimeError: No active exception to reraise

In [15]:
non_triv_count = 0

for day, clusters in data.items():
    for cluster_id, post_ids in clusters.items():
        if len(post_ids) > 1:
            non_triv_count += 1

non_triv_count

3026

In [13]:
same_count = 0

for day, clusters in data.items():
    for cluster_id, post_ids in clusters.items():
        if len(post_ids) > 1:
            posts = data_df.filter(pl.col('id').is_in(post_ids))['title'].to_list()
            if len(set(posts)) == 1:
                same_count += 1

same_count


KeyboardInterrupt



In [ ]:
2143 -> 264
2588 -> 288
3026 -> 264

##### validation samples from clusters

In [4]:
with open(data_dir+'post_clusters.json') as f:
    data = json.load(f)

In [5]:
day_sample = 1000
post_sample = 10000
random.seed(42)

cluster_sample = post_sample // day_sample
post_id_list = []

d = 1
for day, clusters in data.items():
    
    cluster_ids = list(clusters.keys())
    p_ids = []
    
    for id in cluster_ids:
        if len(clusters[id]) > 1:
            p_ids.extend(clusters[id])
            
    day_ids = random.sample(p_ids, cluster_sample)
    post_id_list.extend(day_ids)

    d += 1

    if d > day_sample:
        break

In [6]:
len(post_id_list)

10000

In [7]:
val_dict = {'post_ids' : post_id_list}
with open(data_dir+'sample_post_ids_10k.json', 'w') as f:
    json.dump(val_dict, f)

In [8]:
with open(data_dir+'sample_post_ids_10k.json', 'r') as f:
    val_data = json.load(f)

In [9]:
sample_list = list(val_data.values())[0]

In [10]:
len(sample_list)

10000

In [11]:
all_sample_posts = data_df.filter(pl.col('id').is_in(sample_list))['title'].to_list()

In [12]:
all_sample_posts = all_sample_posts[:post_sample]

In [13]:
len(all_sample_posts)

10000

In [16]:
sample_posts = all_sample_posts

##### chatgpt to find metaphors on the samples

In [14]:
from openai import OpenAI

In [15]:
# gpt3.5 client
client = OpenAI(api_key="api_key")  # "api_key"

In [20]:
sample_posts[18]

"Why Andrew Yang's UBI Is Worth a Look for Dems in 2020"

In [36]:
c_size = 1000
sample_dict = {i:sample_posts[i*c_size:(i+1)*c_size] for i in range(len(sample_posts)//c_size)}

In [46]:
sample_dict[2]

['Geert Wilders is part Indonesian, which is odd in a White Supremacist! [Kidding! Make Holland Great Again!]',
 'Missouri Lt. Gov. Mike Kehoe running for governor, decides against bid for U.S. Senate',
 'Boris Johnson sex dossier sparks Tory civil war',
 "Did Hillary Clinton Tell FBI's Mueller to Deliver Uranium to Russians in 2009 'Secret Tarmac Meeting?'",
 'White House: ‘Ridiculous To Suggest’ Trump Has Hurt Freedom Of The Press',
 'On the doctors plot- The argument that Stalin was anti semitic',
 "Trump cites 'just released' book manuscript to attack Cohen",
 'Since Mueller can’t indict Trump, what’s he doing with a grand jury?',
 'Trump taken to hospital after testing positive',
 'Tolerant liberals harassed a cute girl out of quiting liberal arts college because she dared to publicly support Trump',
 'Florida attorney reportedly under investigation after telling Republicans to change "your address for the next 2 months" for Georgia runoffs',
 'TRUMP CURSE: CBS honcho Les Moonves 

In [54]:
all_responses = []

In [112]:
#sample_posts = all_sample_posts
responses_chunk = []

bar = tqdm(range(c_size))
for post in sample_dict[9]:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_format={ "type": "json_object" },
        seed=20,
        temperature=0.53,
        messages=[
            {
                "role": "system",
                "content": "You will be provided with a Reddit post title, and your task is to identify if the post contains a sports metaphor or not. Note that sports related words may be used in a nonmetaphorical way, do not label such cases as sports metaphors. If the text does contain a sports metaphor, identify the sports metaphor word or phrase and provide a max 10 word explanation. Provide the answer in a JSON format with the following keys, contains_sports_metaphor (true/false), sports_metaphor, explanation."
            },
            {
                "role": "user",
                "content": post
            }
        ],
    )
                
    obj = json.loads(response.json())
    resp_json = json.loads(obj["choices"][0]["message"]["content"])
    resp_json["post"] = post
    responses_chunk.append(resp_json)
    bar.update(1)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [113]:
responses_chunk

[{'contains_sports_metaphor': False,
  'post': 'GOP scrambles to regain fiscal credibility with House budget'},
 {'contains_sports_metaphor': False,
  'sports_metaphor': None,
  'explanation': None,
  'post': 'Time for a History Lesson about Gun Control | Stephen P. Halbrook'},
 {'contains_sports_metaphor': False,
  'post': 'Hillary Clinton Interviews Bernie supporters'},
 {'contains_sports_metaphor': False,
  'sports_metaphor': None,
  'explanation': None,
  'post': 'Former President Jimmy Carter: Trump "is very careless with the truth"'},
 {'contains_sports_metaphor': False,
  'sports_metaphor': None,
  'explanation': None,
  'post': "Trump urges ban on 'bump stocks,' other gun modifiers"},
 {'contains_sports_metaphor': False,
  'sports_metaphor': None,
  'explanation': None,
  'post': 'Unathi Kwaza RT from Ed Smith: Not sure who died and appointed the anc as our moral conscience. Morons like cele, ramaphosa and ndz are taking awuour freedom as well as our freedom of choice! If I wan

In [114]:
all_responses.extend(responses_chunk)

In [83]:
all_responses[1000]

{'contains_sports_metaphor': False, 'post': 'York College Appointments'}

In [115]:
len(all_responses)

10000

In [116]:
meta = []
non_meta = []

for response in all_responses:
    if response['contains_sports_metaphor']:
        meta.append({'post': response['post'], 'meta': response['sports_metaphor']})
    else:
        non_meta.append({'post': response['post']})

In [117]:
len(meta)

201

In [118]:
with open(data_dir+'gpt3_pos_sample.txt', 'w') as f:
    for item in meta:
        f.write('post : {}'.format(item['post']))
        f.write('\n')
        f.write('meta : {}'.format(item['meta']))
        f.write('\n')
        f.write('\n')

In [119]:
len(meta)

201

In [120]:
len(non_meta)

9799

In [121]:
non_meta_sample = non_meta[:201]

In [122]:
with open(data_dir+'gpt3_neg_sample.txt', 'w') as f:
    for item in non_meta_sample:
        f.write('post : {}'.format(item['post']))
        f.write('\n')
        f.write('\n')

In [123]:
non_meta_sample[:3]

[{'post': 'How Donald Trump Bankrupted His Casinos, Left Contractors Unpaid, Ruined Investors &amp; Made Millions (video)'},
 {'post': 'Just getting back from the rally at University of New Hampshire...'},
 {'post': 'Trump privately blamed Black Americans for lacking initiative: report'}]

#### Analyze matched posts

In [5]:
with open('/users/ujan/Downloads/post_cluster_matches_5.json') as f:
    data = json.load(f)

##### total clusters in 5 days

In [6]:
tot = 0

for day, clusters in data.items():
    tot += len(clusters)

tot

2049

##### clusters with metaphors

In [7]:
meta_cluster_count = 0

for day, clusters in data.items():
    for c_id, matches in clusters.items():
        if len(matches['meta']) > 0:
            #print(data_df.filter(pl.col('id').is_in(matches['meta']))['title'].to_list())
            #print('END')
            #print(' ')
            meta_cluster_count += 1

meta_cluster_count

204

##### examples

In [33]:
for day, clusters in data.items():
    for c_id, matches in clusters.items():
        if len(matches['meta']) > 0:
            print(data_df.filter(pl.col('id').is_in(matches['meta']))['title'].to_list())
            print('END')
            print(' ')

['Seattle judge blocks Trump immigration order', "Trump to overturn 'so-called' judge's ban", 'Boston Judge Unblocks Trump Travel Ban, Asks "Where Does It Say Muslim Countries?"', 'Trump questions authority of judge who halted his immigration ban', 'Federal Judge Halts Trump Travel Ban Nationwide']
END
 
['FBI Vault quietly drops another set of Clinton Files - release 6', 'Midnight “Release” Again – 178 Page FBI Clinton Email Document Release With Interesting Content…']
END
 
['Der Spiegel: Trump beheading cover sparks criticism']
END
 
["Judge Who Blocked Trump Action Has Activist Past: Robart made a point of saying 'Black Lives Matter' in a dispute with Seattle police, does pro bono work for refugees.", "Same judge who halted the EO travel ban declared 'black lives matter' from the bench in Seattle @2:18"]
END
 
["Open Letter to President Trump: Constitutionally, the POTUS only has 3 enumerated powers. One charges him with the 'protection' of this sovereign nation. I know you don't h

KeyboardInterrupt: 

#### Num comments in meta posts vs non meta posts in same cluster

In [31]:
for day, clusters in data.items():
    for c_id, matches in clusters.items():
        if len(matches['meta']) > 0:
            meta_num_comments = data_df.filter(pl.col('id').is_in(matches['meta']))['num_comments'].to_list()
            non_meta_num_comments = data_df.filter(pl.col('id').is_in(matches['non_meta']))['num_comments'].to_list()
            print(statistics.median(meta_num_comments))
            print(statistics.median(non_meta_num_comments))
            print(' ')

5
5
 
61.0
20.0
 
17
7.0
 
58.0
7
 
5
14.5
 
4
2
 
41
6.0
 
23
6.5
 
11
21
 
1485.0
5
 
19.0
5
 
17
5
 
2
4.0
 
27
3
 
50
42
 
8
4
 
16


StatisticsError: no median for empty data

##### median num_comments

In [8]:
meta_num_comments = []
non_meta_num_comments = []

for day, clusters in data.items():
    for c_id, matches in clusters.items():
        if len(matches['meta']) > 0 and len(matches['non_meta']) > 0:
            meta_num_comments.extend(data_df.filter(pl.col('id').is_in(matches['meta']))['num_comments'].to_list())
            non_meta_num_comments.extend(data_df.filter(pl.col('id').is_in(matches['non_meta']))['num_comments'].to_list())
            
print(statistics.median(meta_num_comments))
print(statistics.median(non_meta_num_comments))

7
7.0


##### mean num_comments

In [10]:
print(statistics.mean(meta_num_comments))
print(statistics.mean(non_meta_num_comments))

39.327510917030565
52.67888888888889


##### median comparison across clusters

In [11]:
meta_median_c = 0
non_meta_median_c = 0

for day, clusters in data.items():
    for c_id, matches in clusters.items():
        if len(matches['meta']) > 0 and len(matches['non_meta']) > 0:
            meta_num_comments = data_df.filter(pl.col('id').is_in(matches['meta']))['num_comments'].to_list()
            non_meta_num_comments = data_df.filter(pl.col('id').is_in(matches['non_meta']))['num_comments'].to_list()
            meta_median = statistics.median(meta_num_comments)
            non_meta_median = statistics.median(non_meta_num_comments)

            if meta_median > non_meta_median :
                meta_median_c += 1
            elif non_meta_median > meta_median:
                non_meta_median_c += 1

print(meta_median_c)
print(non_meta_median_c)

83
83


##### only popular subs

In [12]:
sublist = ["politics", "The_Donald", "Conservative", "PoliticalDiscussion", "AskTrumpSupporters", "AskALiberal", "NeutralPolitics", "neutralnews", "moderatepolitics", "worldnews", "Republican", "Liberal", "progressive", "esist"]

In [15]:
pol_ids = data_df.filter(pl.col('subreddit').is_in(sublist))['id'].to_list()

##### median num_comments

In [26]:
meta_num_comments = []
non_meta_num_comments = []

for day, clusters in data.items():
    for c_id, matches in clusters.items():

        if all(item in pol_ids for item in matches['meta']) and all(item in pol_ids for item in matches['non_meta']):
        
            if len(matches['meta']) > 0 and len(matches['non_meta']) > 0:
                meta_num_comments.extend(data_df.filter(pl.col('id').is_in(matches['meta']))['num_comments'].to_list())
                non_meta_num_comments.extend(data_df.filter(pl.col('id').is_in(matches['non_meta']))['num_comments'].to_list())
            
print(statistics.median(meta_num_comments))
print(statistics.median(non_meta_num_comments))

8
8


In [28]:
print(statistics.mean(meta_num_comments))
print(statistics.mean(non_meta_num_comments))

33.16438356164384
29.63157894736842
